In [1]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import numpy as np
import os, glob, cv2, torch
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# 设置设备
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    
    
data_dir = './videos/CT_tt'
video_dir = os.path.join(data_dir, 'img_in_jpg')
target_dir = os.path.join(data_dir, 'img_in_jpg_2class_DINO_order')


os.makedirs(target_dir, exist_ok=True)

# 加载 DINO 预训练模型
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-large')
model = AutoModel.from_pretrained('facebook/dinov2-large')
model.to(device)
model.eval()

# 处理图像
def process_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    return inputs

# 计算图像特征向量
def get_image_embedding(image_path):
    inputs = process_image(image_path)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:

# 获取所有图片路径
all_image_files = sorted(glob.glob(os.path.join(video_dir, "**/*.png"), recursive=True) +
                         glob.glob(os.path.join(video_dir, "**/*.jpg"), recursive=True))

# 计算所有图片的嵌入向量
image_embeddings = []
for image_path in all_image_files:
    embedding = get_image_embedding(image_path)
    image_embeddings.append(embedding)

# 转换为 NumPy 数组
image_embeddings = np.vstack(image_embeddings)

# 计算相似度矩阵
similarity_matrix = cosine_similarity(image_embeddings)

# -------------- K-Means 聚类 --------------
num_clusters = 3  # 设定核心数
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(image_embeddings)

# -------------- 找出每个簇的核心图像 --------------
cluster_centers = kmeans.cluster_centers_
core_images = []

for cluster_id in range(num_clusters):
    cluster_indices = np.where(cluster_labels == cluster_id)[0]  # 获取该簇的所有图像索引
    cluster_embeddings = image_embeddings[cluster_indices]

    # 计算该簇内所有图片到簇中心的欧式距离
    distances_to_center = np.linalg.norm(cluster_embeddings - cluster_centers[cluster_id], axis=1)
    
    # 找到距离簇中心最近的图片索引
    core_index = cluster_indices[np.argmin(distances_to_center)]
    core_images.append(core_index)

# -------------- 按照核心图像距离进行排序 --------------
remaining_indices = set(range(len(all_image_files))) - set(core_images)

# 计算每张图像到 3 个核心图像的总距离
distance_sums = {}
for idx in remaining_indices:
    distance_sum = sum(np.linalg.norm(image_embeddings[idx] - image_embeddings[core], axis=0) for core in core_images)
    distance_sums[idx] = distance_sum

# 按距离和排序（距离大 → 靠后）
sorted_remaining_indices = sorted(distance_sums.keys(), key=lambda x: distance_sums[x])

# 组合排序后的索引
sorted_indices = core_images + sorted_remaining_indices

# -------------- 保存排序后的图片 --------------
for i, idx in enumerate(sorted_indices):
    image_path = all_image_files[idx]
    image = cv2.imread(image_path)
    output_path = os.path.join(target_dir, f"{i:05d}_{os.path.basename(image_path)}")
    print(f"{i}: {output_path}")

    cv2.imwrite(output_path, image)

print(f"排序后的图片已保存到 {target_dir}")


0: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00000_s1355_img_L4L5_slice134.jpg
1: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00001_s0476_img_L4L5_slice151.jpg
2: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00002_s0034_img_L4L5_slice172.jpg
3: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00003_s0863_img_L4L5_slice158.jpg
4: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00004_s0636_img_L4L5_slice139.jpg
5: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00005_s1377_img_L4L5_slice148.jpg
6: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00006_s0472_img_L4L5_slice139.jpg
7: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00007_s0549_img_L4L5_slice159.jpg
8: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00008_s0644_img_L4L5_slice148.jpg
9: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00009_s0402_img_L4L5_slice155.jpg
10: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00010_s0108_img_L4L5_slice144.jpg
11: ./videos/CT_tt/img_in_jpg_2class_DINO_order/00011_s1382_img_L4L5_slice160.jpg
12: ./videos/CT_tt/img_in_

In [11]:
import os
import shutil
import glob

# 定义文件夹路径
import re

def modify_filename(base_name):
    # 替换 img 为 label
    new_name = re.sub(r'img', 'label', base_name)
    # 替换 .jpg 为 .png（只处理结尾）
    new_name = re.sub(r'\.jpg$', '.png', new_name)
    return new_name


# video_dir = os.path.join(data_dir, 'img_in_jpg_' + level  + '_4class_DINO_order')
# label_dir  =  os.path.join(data_dir, 'label_in_png_' + level  + '_4class')
# target_label_dir = os.path.join(data_dir, 'label_in_png_' + level  + '_4class_DINO_order')

video_dir = os.path.join(data_dir, 'img_in_jpg_2class_DINO_order')
label_dir  =  os.path.join(data_dir, 'label_in_png')
target_label_dir = os.path.join(data_dir, 'label_in_png_2class_DINO_order')

os.makedirs(target_label_dir, exist_ok=True)

# 获取视频文件列表
video_files = sorted(glob.glob(os.path.join(video_dir, "*.jpg")))

# 遍历文件名提取ID，并处理对应的标签文件
for video_file in video_files:
    # 获取文件名（如 "00001_GL_106_slice_016_img_.jpg"）
    base_name = os.path.basename(video_file)
    
    # 提取ID部分（如 "GL_106"）
    parts = base_name.split('_')
    id_part = "_".join(parts[1:3])  # 提取 "GL_106"
    if 'CT_tt' in label_dir:
        parts = base_name.split('_')
        id_part = "_".join(parts[1:2])  # 提取 "GL_106"
    print('pid',id_part)
    # 查找标签文件
    label_file_pattern = os.path.join(label_dir, f"*{id_part}*.png")
    label_files = glob.glob(label_file_pattern)
    
    if len(label_files) == 0:
        print(f"未找到匹配的标签文件: {id_part}")
        continue
    
    # 如果找到多个匹配，提示警告（根据需要可以调整逻辑）
    if len(label_files) > 1:
        print(f"找到多个匹配的标签文件: {label_files}")
    
    # 取第一个匹配的文件
    label_file = label_files[0]
    
    # 生成目标文件名，保持与视频文件一致的格式
    target_label_file = os.path.join(target_label_dir, base_name.replace('img.jpg', 'seg_.png'))
    if 'CT_tt' in label_dir:
        target_label_file = os.path.join(target_label_dir, modify_filename(base_name))
    # 复制文件到目标文件夹
    shutil.copy(label_file, target_label_file)

    print(f"处理完成: {label_file} -> {target_label_file}")
    print('\n\n\n')

print(f"所有标签文件已保存到 {target_label_dir}")


pid s1355
处理完成: ./videos/CT_tt/label_in_png/s1355_png_L4L5_slice134.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/00000_s1355_label_L4L5_slice134.png




pid s0476
处理完成: ./videos/CT_tt/label_in_png/s0476_png_L4L5_slice151.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/00001_s0476_label_L4L5_slice151.png




pid s0034
处理完成: ./videos/CT_tt/label_in_png/s0034_png_L4L5_slice172.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/00002_s0034_label_L4L5_slice172.png




pid s0863
处理完成: ./videos/CT_tt/label_in_png/s0863_png_L4L5_slice158.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/00003_s0863_label_L4L5_slice158.png




pid s0636
处理完成: ./videos/CT_tt/label_in_png/s0636_png_L4L5_slice139.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/00004_s0636_label_L4L5_slice139.png




pid s1377
处理完成: ./videos/CT_tt/label_in_png/s1377_png_L4L5_slice148.png -> ./videos/CT_tt/label_in_png_2class_DINO_order/00005_s1377_label_L4L5_slice148.png




pid s0472
处理完成: ./videos/CT_tt/lab

In [5]:
import os
import glob

# 定义文件夹路径
# data_dir = './videos/AFL_MRI/AFL_L4L5_in_JPG_2class_DINO'

label_dir = target_label_dir 

# 获取所有视频文件和标签文件
video_files = sorted(glob.glob(os.path.join(video_dir, "*.jpg")))
label_files = sorted(glob.glob(os.path.join(label_dir, "*.png")))

# 检查数量是否一致
if len(video_files) != len(label_files):
    print(f"文件数量不匹配: 视频文件({len(video_files)}) 与 标签文件({len(label_files)})")
else:
    print(f"文件数量匹配: {len(video_files)} 个文件")

# 提取文件名中的 ID
def extract_id(file_name):
    base_name = os.path.basename(file_name)
    parts = base_name.split('_')
    return "_".join(parts[1:3])  # 提取 ID，如 "GL_106"

# 对比两个文件夹的 ID
mismatch_count = 0
for video_file, label_file in zip(video_files, label_files):
    video_id = extract_id(video_file)
    label_id = extract_id(label_file)
    print(video_id, label_id)
    
    if video_id != label_id:
        print(f"不匹配: 视频文件 {video_file} 的 ID ({video_id}) 与 标签文件 {label_file} 的 ID ({label_id}) 不一致")
        mismatch_count += 1

# 总结结果
if mismatch_count == 0:
    print("所有文件的 ID 均匹配！")
else:
    print(f"总共有 {mismatch_count} 个文件的 ID 不匹配。")


文件数量匹配: 288 个文件
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0010_img_L4L5_slice130.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0010_label_L4L5_slice130.png 的 ID (label_L4L5) 不一致
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0011_img_L4L5_slice140.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0011_label_L4L5_slice140.png 的 ID (label_L4L5) 不一致
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0014_img_L4L5_slice135.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0014_label_L4L5_slice135.png 的 ID (label_L4L5) 不一致
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0015_img_L4L5_slice147.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0015_label_L4L5_slice147.png 的 ID (label_L4L5) 不一致
img_L4L5 label_L4L5
不匹配: 视频文件 ./videos/CT_tt/img_in_jpg/s0016_img_L4L5_slice142.jpg 的 ID (img_L4L5) 与 标签文件 ./videos/CT_tt/label_in_png_2class_DINO_order/s0016_label